In [1]:
%run utils/devtools.py

In [2]:
%reloadmypkg utils

import pandas as pd
from utils.url2platform import *
from utils.standardise_url import *
from utils.fetch_info import *
from utils.dump_df_to_json import *

✅ Reloaded package 'utils' and its submodules.


In [3]:
sheet_name = "../raw-data/leetcode/150-response-data.json"
export_file = "../cleaned-data/leetcode/150.json"

In [4]:
with open(sheet_name) as file:
    raw = json.load(file)

flattened_data = [
    "https://leetcode.com/problems/" + question["titleSlug"]
    for subgroup in raw["data"]["studyPlanV2Detail"]["planSubGroups"]
    for question in subgroup["questions"]
]

df = pd.DataFrame(flattened_data, columns=['url'])

In [5]:
df.head(2)

,url
0,https://leetcode.com/problems/merge-sorted-array
1,https://leetcode.com/problems/remove-element


In [6]:
df.shape

(150, 1)

In [7]:
# Find the duplicates

dupes_df = df[df.duplicated(subset="url", keep=False)]
dupes_df.shape

(0, 1)

In [8]:
import nest_asyncio
nest_asyncio.apply()

standardised_df = await standardise_urls_async(df, "leetcode-150-urls-expanded.json")

Filter list is empty. Trying to expand all 150 URLs.

Processing batch 1/1
Loaded 0 results from checkpoint
Processing 150 pending URLs in this batch


Expanding URLs: 100%|██████████| 150/150 [00:01<00:00, 95.23it/s] 

Progress saved to leetcode-150-urls-expanded.json
----------------------------------------------------------------------
Progress saved to leetcode-150-urls-expanded.json
----------------------------------------------------------------------

150/150 URLs expanded in 3.95 seconds.


In [9]:
standardised_df.shape

(150, 3)

In [10]:
# Adding type column
standardised_df['type'] = standardised_df['expanded_stripped_url'].apply(
    lambda url: "problem" if is_problem_url(url) else "tutorial"
)

In [11]:
# Adding id_base, id, title, platform columns
import nest_asyncio
nest_asyncio.apply()

final_df = await fetch_info(standardised_df, "leetcode-150-info-added.json", resume=True)

Found 150 URLs.

Batch 1/1: items 0-149
Loaded 0 from checkpoint
Processing 150 URLs...


Fetching items: 100%|██████████| 150/150 [00:03<00:00, 38.42it/s] 

Saved progress (150 items) to leetcode-150-info-added.json
------------------------------
Saved progress (150 items) to leetcode-150-info-added.json
------------------------------

Fetched 150/150 items in 3.9s.


In [12]:
final_df.shape

(150, 8)

In [13]:
dump_df_to_json(final_df, export_file)